# IMPORTS

In [51]:
import os
import torch
import torchaudio

In [52]:
import sounddevice as sd

In [90]:
os.listdir('./save/state')[-10]

'ss_30.pt'

In [91]:
from importlib.machinery import SourceFileLoader
mi_clasificador = SourceFileLoader('clasificador_padre', '../libs/clasificador_padreV2.py').load_module()
folder_path = "./save/state/"


In [92]:
version = os.listdir(folder_path)[-1]
model_path = os.path.join(folder_path, version)

In [93]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

CARGO EL MODELO

In [94]:
#Metaparameters for MFCC transformer
#parametros para los calculos del mell
TARGET_SR = 16000  # Normalmente el audio se sule usar a 16k aunque encontre papers que trabajan a 22050 o a 22k (ver V2)
N_FFT = 1024  #muestras de la fft
W_LEN = 800  # Numero de muestras para la ventan de la  fft (seg_de_ventan *sr) 
H_LEN = 320 # paso de la ventana entre una fft y la siguiente (paso * sr)
N_MELS = 40#26
N_MFCC = 32#13

MFCCCalculator = torchaudio.transforms.MFCC(sample_rate = TARGET_SR,
                                            n_mfcc = int(N_MFCC),
                                            dct_type = 2,
                                            norm = 'ortho',
                                            log_mels = False,
                                            melkwargs = 
                                            {
                                                "n_fft": N_FFT,          # Size of FFT (2048)
                                                "win_length": W_LEN,     # Actual window size (400 samples = 25ms)
                                                "hop_length": H_LEN,     # Hop length (160 samples = 10ms)
                                                "n_mels": N_MELS,        # Number of Mel bins (40)
                                                "center": False
                                                },)

In [95]:
def log_weight_norms(model, epoch):
    total_l2_norm_sq = 0.0
    print(f"\nEpoch {epoch+1} - Weight L2 Norms:")
    for name, param in model.named_parameters():
        if param.requires_grad and "weight" in name: # Comúnmente los pesos se llaman 'weight'
            norm = torch.norm(param.data, p=2) # Calcula la norma L2
            #print(f"  Layer '{name}': {norm.item():.4f}")
            total_l2_norm_sq += norm.item()**2
        # Podrías añadir un chequeo para 'bias' si también te interesan

    overall_norm = total_l2_norm_sq**0.5
    print(f"  Overall Network L2 Norm (Weights Only): {overall_norm:.4f}")
    # Puedes guardar este valor en una lista para graficarlo después
    return overall_norm

In [96]:
#{'Ariana Grande': tensor([1., 0., 0., 0., 0.]), 'Katy Perry': tensor([0., 1., 0., 0., 0.]), 'Metallica': tensor([0., 0., 1., 0., 0.]), 'Pink Floyd': tensor([0., 0., 0., 1., 0.]), 'Taylor Swift': tensor([0., 0., 0., 0., 1.])}

In [97]:
clasificador = mi_clasificador.onlyWoman_MFCC_16k_v8(['Ariana Grande', 'Katy Perry', 'Metallica', 'Pink Floyd', 'Taylor Swift'], MFCCCalculator).to(device)
clasificador.load_state_dict(torch.load(model_path))
clasificador.eval()

onlyWoman_MFCC_16k_v8(
  (transformer): MFCC(
    (amplitude_to_DB): AmplitudeToDB()
    (MelSpectrogram): MelSpectrogram(
      (spectrogram): Spectrogram()
      (mel_scale): MelScale()
    )
  )
  (inPut): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (chanelUp): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (justTime): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1,

#PARAMETROS PARA PROCESAR EL AUDIO

In [98]:
dic = {0:'Ariana Grande', 1:'Katy Perry', 2:'Metallica', 3:'Pink Floyd', 4:'Taylor Swift'}

In [103]:
record = sd.rec(int(10*TARGET_SR), samplerate=TARGET_SR, channels=1, dtype='float32', blocking= True)
record_tensor = torch.from_numpy(record.T).to(device)
#record_tensor = record_tensor / record_tensor.abs().max()
record_tensor= clasificador.audio_norm(record_tensor)
#mfcc = MFCCCalculator(record_tensor)
clasificador.eval()
res = clasificador(record_tensor)
pred = dic[int(torch.argmax(res))]
print(pred)
print(res)

Ariana Grande
tensor([[0.5105, 0.4409, 0.0153, 0.0161, 0.0171]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)


In [102]:
res

tensor([[0.0367, 0.9397, 0.0075, 0.0079, 0.0083]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)

# eval de pesos

In [87]:
for i in range(len(os.listdir(folder_path))):
    patito = os.path.join(folder_path, os.listdir(folder_path)[i])
    clasificador.load_state_dict(torch.load(patito))
    log_weight_norms(clasificador, i)


Epoch 1 - Weight L2 Norms:
  Overall Network L2 Norm (Weights Only): 15.3236

Epoch 2 - Weight L2 Norms:
  Overall Network L2 Norm (Weights Only): 13.7884

Epoch 3 - Weight L2 Norms:
  Overall Network L2 Norm (Weights Only): 12.9787

Epoch 4 - Weight L2 Norms:
  Overall Network L2 Norm (Weights Only): 12.3871

Epoch 5 - Weight L2 Norms:
  Overall Network L2 Norm (Weights Only): 11.8943

Epoch 6 - Weight L2 Norms:
  Overall Network L2 Norm (Weights Only): 11.4483

Epoch 7 - Weight L2 Norms:
  Overall Network L2 Norm (Weights Only): 11.0435

Epoch 8 - Weight L2 Norms:
  Overall Network L2 Norm (Weights Only): 10.6748

Epoch 9 - Weight L2 Norms:
  Overall Network L2 Norm (Weights Only): 10.3452

Epoch 10 - Weight L2 Norms:
  Overall Network L2 Norm (Weights Only): 10.0488

Epoch 11 - Weight L2 Norms:
  Overall Network L2 Norm (Weights Only): 9.7898

Epoch 12 - Weight L2 Norms:
  Overall Network L2 Norm (Weights Only): 9.5693

Epoch 13 - Weight L2 Norms:
  Overall Network L2 Norm (Weights